### Data pre processing

In [ ]:
import numpy as np
obj_centre = []
obj_width = []
lines = []
lines = [line.rstrip('\n') for line in open("./cifar10_transformed/devkit/train.txt")]
obj_centre = []
obj_width = []


lines = np.asarray(lines)
print(lines[0])
for i in range(len(lines)):
    obj_centre.append([lines[i][12:15],lines[i][15:18]])
    obj_width.append(lines[i][18:])

obj_centre = np.asarray(obj_centre)
obj_width = np.asarray(obj_width)
print(obj_centre.shape)
print(obj_width.shape)


In [ ]:
import glob
import numpy as np
from scipy import misc
files = glob.glob("./cifar10_transformed/masks/*.png")
masks = []
for i in range(len(files)):
    temp_mask = misc.imread(files[i])
    masks.append(temp_mask)
masks_reshaped = []
for i in range(len(masks)):
    masks_reshaped.append(np.ravel(masks[i]))

In [ ]:
data = []
file_data = glob.glob("./cifar10_transformed/imgs/*.png")
for i in range(len(file_data)):
    name = file_data[i]
    data.append(misc.imread(name))

data = np.asarray(data)
print(data.shape)

In [ ]:
import numpy as np
lines = [line.rstrip('\n') for line in open("./cifar10_transformed/devkit/train.txt")]
rows = []
columns = []
width = []


lines = np.asarray(lines)
print(lines[0])
for i in range(len(lines)):
    rows.append(lines[i][12:15])
    columns.append(lines[i][15:18])
    width.append(lines[i][18:])

In [ ]:
dimensions = np.vstack([rows,columns,width])
print(dimensions.shape)
dimensions = dimensions.transpose()
print(dimensions[1000][:])

In [ ]:
import numpy as np
lines = [line.rstrip('\n') for line in open("./cifar10_transformed/devkit/test.txt")]
rows = []
columns = []
width = []


lines = np.asarray(lines)
print(lines[0])
for i in range(len(lines)):
    rows.append(lines[i][12:15])
    columns.append(lines[i][15:18])
    width.append(lines[i][18:])

In [ ]:
dimensions_test = np.vstack([rows,columns,width]).transpose()
print(dimensions_test.shape)

In [ ]:
import tensorflow as tf
x = tf.placeholder(tf.float32, shape=[None, 48,48,3])

def batch_norm(x, is_train=True, decay=0.99, epsilon=0.001):
    shape_x = x.get_shape().as_list()
    beta = tf.get_variable('beta', shape_x[-1], initializer=tf.constant_initializer(0.0))
    gamma = tf.get_variable('gamma', shape_x[-1], initializer=tf.constant_initializer(1.0))
    moving_mean = tf.get_variable('moving_mean', shape_x[-1],
                initializer=tf.constant_initializer(0.0), trainable=False)
    moving_var = tf.get_variable('moving_var', shape_x[-1],
               initializer=tf.constant_initializer(1.0), trainable=False)

    if is_train:
        mean, var = tf.nn.moments(x, np.arange(len(shape_x)-1), keep_dims=True)
        mean = tf.reshape(mean, [mean.shape.as_list()[-1]])
        var = tf.reshape(var, [var.shape.as_list()[-1]])

        update_moving_mean = tf.assign(moving_mean, moving_mean*decay + mean*(1-decay))
        update_moving_var = tf.assign(moving_var,
                            moving_var*decay + shape_x[0]/(shape_x[0]-1)*var*(1-decay))
        update_ops = [update_moving_mean, update_moving_var]

        with tf.control_dependencies(update_ops):
            return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

    else:
        mean = moving_mean
        var = moving_var
        return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
slim = tf.contrib.slim
mu = 0
sigma = 0.1

img_index = 0
def cifarNet(x, y, dims, is_train=True):
    two = tf.constant(2,tf.float32)
    one = tf.constant(1,tf.float32)
    vs = tf.get_variable_scope()
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='SAME') + conv1_b
    conv1 = batch_norm(conv1)
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 2
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(64))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='SAME') + conv2_b
    conv2 = batch_norm(conv2)
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 3
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='SAME') + conv3_b
    
    conv3 = batch_norm(conv3)
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 4
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 128, 256), mean = mu, stddev = sigma))
    conv4_b = tf.Variable(tf.zeros(256))
    
    conv4   = tf.nn.conv2d(conv3, conv4_W, strides=[1, 1, 1, 1], padding='SAME') + conv4_b
    conv4 = batch_norm(conv4)
    
    conv4 = tf.nn.relu(conv4)
    
    #Layer 5 intermediate layer
    conv5_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 256, 256), mean = mu, stddev = sigma))
    conv5_b = tf.Variable(tf.zeros(256))
    
    conv5   = tf.nn.conv2d(conv4, conv5_W, strides=[1, 1, 1, 1], padding='SAME') + conv5_b
    conv5 = batch_norm(conv5)
    conv5 = tf.nn.relu(conv5)
    
    #Layer 6 for classification
    conv6_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 256, 1), mean = mu, stddev = sigma))
    conv6_b = tf.Variable(tf.zeros(1))
    
    conv6   = tf.nn.conv2d(conv5, conv6_W,strides=[1, 1, 1, 1], padding='SAME') + conv6_b
    conv6 =  flatten(conv6)
   
    bool_indices = tf.not_equal(y, two)
    threshold_indices = tf.where(bool_indices)
    ground_truth = tf.gather_nd(y, threshold_indices)
    predicted = tf.gather_nd(conv6,threshold_indices)
    
    #Layer 7 for regression
    conv7_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 256, 3), mean = mu, stddev = sigma))
    conv7_b = tf.Variable([24,24,32], dtype=tf.float32)
    
    conv7   = tf.nn.conv2d(conv5, conv7_W,strides=[1, 1, 1, 1], padding='SAME') + conv7_b
    

    
    bool_indices_ones = tf.equal(y, one)
    threshold_indices_ones = tf.where(bool_indices_ones)
    
    conv7 = tf.reshape(conv7,[100,36,3])
    predicted_rows = tf.slice(conv7,[0,0,0], [100,36,1])
    predicted_rows = tf.reshape(predicted_rows,[100,36])

    predicted_columns = tf.slice(conv7,[0,0,1],[100,36,1])
    predicted_columns = tf.reshape(predicted_columns,[100,36])

    
    predicted_width = tf.slice(conv7,[0,0,2],[100,36,1])
    predicted_width = tf.reshape(predicted_width,[100,36])

    
    predicted_rows_dimensions = tf.gather_nd(predicted_rows, threshold_indices_ones)
    predicted_columns_dimensions = tf.gather_nd(predicted_columns, threshold_indices_ones)
    predicted_width_dimensions = tf.gather_nd(predicted_width, threshold_indices_ones)
    
    anchor_size = tf.constant(32,dtype=tf.float32)
    predicted_rows_dimensions = predicted_rows_dimensions / anchor_size

    predicted_rows_dimensions = tf.reshape(predicted_rows_dimensions,[-1,1])
    
    predicted_columns_dimensions = predicted_columns_dimensions / anchor_size
    predicted_columns_dimensions = tf.reshape(predicted_columns_dimensions,[-1,1])

    
    predicted_width_dimensions = predicted_width_dimensions / anchor_size
    predicted_width_dimensions = tf.reshape(predicted_width_dimensions,[-1,1])

    

    row_dims = tf.slice(dims,[0,0],[100, 1])
    row_dims = tf.tile(row_dims,[1,36])
    rows_dims = tf.gather_nd(row_dims, threshold_indices_ones)
    rows_dims = rows_dims / anchor_size
    rows_dims = tf.reshape(rows_dims,[-1,1])
    
    
    column_dims = tf.slice(dims,[0,1],[100, 1])
    column_dims = tf.tile(column_dims,[1,36])
    columns_dims = tf.gather_nd(column_dims, threshold_indices_ones)
    columns_dims = columns_dims / anchor_size
    columns_dims = tf.reshape(columns_dims,[-1,1])
    
    w_dims = tf.slice(dims,[0,2],[100, 1])
    w_dims = tf.tile(w_dims,[1,36])
    width_dims = tf.gather_nd(w_dims, threshold_indices_ones)
    width_dims = width_dims / anchor_size
    width_dims = tf.reshape(width_dims,[-1,1])
    
    ground_dims = tf.concat([rows_dims, columns_dims,width_dims], axis=1)
    

    predicted_dimensions = tf.concat([predicted_rows_dimensions, predicted_columns_dimensions, predicted_width_dimensions],axis=1)


    
    return conv6, ground_truth,predicted, predicted_dimensions, ground_dims
    

In [ ]:
from sklearn.utils import shuffle

BATCH_SIZE = 100
EPOCHS = 2000
rate_initial = 0.001
is_train = True
batch = tf.Variable(0)
losses_training = []
y = tf.placeholder(tf.float32, shape=[None,36])
w = tf.placeholder(tf.float32, shape=[None,6,6])
dims = tf.placeholder(tf.float32, shape=[None,3])
ground_dimensions = tf.placeholder(tf.float32, shape=[None,6,3])
logits, ground_truth ,predicted, predicted_dimensions,ground_dims = cifarNet(x, y,dims, is_train)


predicted_temp = tf.nn.sigmoid(predicted)
predicted_threshold = tf.greater(predicted_temp, 0.5)
predicted_threshold = tf.cast(predicted_threshold, tf.int32)
ground_truth_acc = tf.nn.sigmoid(ground_truth)
predicted_threshold = tf.cast(predicted_threshold, tf.float32)


cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=ground_truth, logits=predicted)

dimensions_loss = tf.losses.huber_loss(predicted_dimensions, ground_dims)


loss_operation = tf.reduce_mean(cross_entropy + dimensions_loss)
rate = tf.train.exponential_decay(rate_initial, batch*BATCH_SIZE,
                                           20, 0.05, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(predicted_dimensions, ground_dims)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.int64))


saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = 10000
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        accuracy = 0
        X_train, y_train, d_train = (data, masks_reshaped, dimensions)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y, batch_d= X_train[offset:end], y_train[offset:end], d_train[offset:end]
            
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, dims:batch_d})
            
            accuracy = accuracy + sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,dims:batch_d})
            break
        print("EPOCH {} ...".format(i+1))
        print("loss")
        temp_loss = sess.run(loss_operation,feed_dict={x: batch_x, y: batch_y, dims:batch_d})
        print(temp_loss)
        losses_training.append(temp_loss)
        print()

    print("testing accuracy")
    print(sess.run(loss_operation,feed_dict={x: data[num_examples+1:], y: masks_reshaped[num_examples+1:], dims:dimensions_test}))
        
    saver.save(sess, './q22Model')
    print("Model saved")